In [44]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, time
import calendar

class Agenda:

   def __init__(self, nombre, datos = pd.DataFrame([])):
      self.nombre = nombre
      self.csv = nombre+'.csv'
      self.__guardadEnCsv(datos)

   def __guardadEnCsv(self, datos):
      datos.to_csv(self.csv)

   def leerDatos(self):
      return pd.read_csv(self.csv, index_col=0)
   
   def mostrarDatos(self):
      print(self.leerDatos())
   
   def insertarDatos(self, datos):
      df = self.leerDatos()
      df = pd.concat([df, datos], axis=0, ignore_index=True)
      self.__guardadEnCsv(df)
   
   def columnasDias(self):
      df = self.leerDatos()
      df = df.columns.values[3:]
      return list(df)

   def guardarTurno(self, id_espe, paciente, dia, horario):
      df = self.leerDatos()
      df.loc[(df['id_espe']) & (df['Horario']==horario), dia] = 'cambiado'
      print(df)

class Especialidad:
   diasDeLaSemana = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
   horaApertura = '10:00'
   horaCierre = '17:00'

   def __init__(self, nombre, id, medico, duracionTurno):
      self.nombre = nombre
      self.id = id
      self.medico = medico
      self.duracionTurno = duracionTurno
      self.__dias = []

   def establecerDias(self, diaInt = 0):
      if diaInt == 0:
         print("Debe establecer los dias de atencion")
         for i in range(len(self.diasDeLaSemana)):
            print (i+1,'-', self.diasDeLaSemana[i])
         print("7 - Lunes a viernes")
         print("8 - TODOS")
         print("0 - SALIR")

         diaInt = int(input("Por favor seleccione los días de atencion de la especialidad, o 0 (cero) para salir"))

      elif (diaInt == 7):
         dias = list(range(0,5))
      elif (diaInt == 8):
         dias = list(range(0,6))
      elif (isinstance(diaInt, list)):
         dias = [x-1 for x in diaInt]
      else:
         dias = []
         while diaInt != 0:
            dias.append(diaInt-1)
            diaInt = int(input("Por favor seleccione los días de atencion de la especialidad, o 0 (cero) para salir"))

      self.__dias = dias
      
   def __formatSemana(self, anio, mes, semana):
      calen = calendar.Calendar()
      semana = calen.monthdayscalendar(anio, mes)[semana]
      diaDelaSemana = self.diasDeLaSemana
      dias = []
      for i in self.__dias:
         dias.append(diaDelaSemana[i]+' '+str(semana[i])+'/'+str(mes))
      return dias

   def establecerTurnos(self, mes, semana, dias = 0):
      anio = 2021

      if (len(self.__dias) == 0):
         self.establecerDias(dias)

      self.diasDisponibles = self.__formatSemana(anio, mes, semana)
      
      if not len(self.diasDisponibles) == 0:
         encabezados = ['id_espe','Especialidad','Horario']+self.diasDisponibles
         inicio = datetime.datetime.strptime(self.horaApertura, '%H:%M')
         turno = datetime.datetime.strptime(self.duracionTurno, '%H:%M')
         time_zero = datetime.datetime.strptime('00:00', '%H:%M')
         final = datetime.datetime.strptime(self.horaCierre, '%H:%M')
         
         final = final - turno + time_zero
         
         filas = []
         while inicio <= final:
            fila = [self.id, self.nombre, inicio.time()]+['disponible' for x in range(len(self.diasDisponibles))]
            inicio = inicio - time_zero + turno
            filas.append(fila)

         df = pd.DataFrame(filas, columns=encabezados)
         #print(df)

      self.turnero.insertarDatos(df)

   def traerDisponibles(self):
      df = self.turnero.leerDatos()
      df = df[df['id_espe'].eq(self.id)]
      df = df.loc[:, ['Horario']+self.diasDisponibles]
      return df

   def asignarTurno(self):
      disponibles = self.traerDisponibles()
      dias = disponibles.columns.values[1:]
      for i in range(len(dias)):
         print (str(i+1),'-',dias[i])
      
      dia = int(input("Selecciona un día para el turno (ingresa el numero correspondiente a la opcion"))

      diaElegido = dias[dia-1]

      horariosDisponibles = self.horario(diaElegido)
      print(horariosDisponibles)

      horarioElegido = int(input("Selecciona el horario: ingresa el numero correspondiente a la opcion"))

      horarioElegido = horariosDisponibles.loc[0, 'Horario']
      print(horarioElegido)

      self.turnero.guardarTurno(self.id, 'paciente', diaElegido, horarioElegido)

   




   def horario(self, dia):
      df = self.turnero.leerDatos()
      df = df[df['id_espe'].eq(self.id)]
      df = df[['Horario']][df[dia] == 'disponible']
      return df.reset_index(drop=True)


odonto = Especialidad('Odontología', 1, 'Luciana Altamirano', '01:00')
oftalmo = Especialidad('Oftalmología', 2, 'Roberto Garcia', '00:20')
gineco = Especialidad('Ginecología', 3, 'Pedro Menuan', '00:30')
clinico = Especialidad('Médico Clínico', 4, 'Alejandra Ilgaramo', '00:25')

turnero = Agenda('turnero')

especialidades = [odonto, oftalmo, gineco, clinico]
for especialidad in especialidades:
   especialidad.turnero = turnero
   especialidad.establecerTurnos(11,3,7)

pediatra = Especialidad('Pediatría', 5, 'Analia Montalvan', '00:40')
pediatra.turnero = turnero
pediatra.establecerTurnos(11,3,[1,3,5])
pediatra.asignarTurno()




1 - Lunes 22/11
2 - Miercoles 24/11
3 - Viernes 26/11
    Horario
0  10:00:00
1  10:40:00
2  11:20:00
3  12:00:00
4  12:40:00
5  13:20:00
6  14:00:00
7  14:40:00
8  15:20:00
9  16:00:00
10:00:00
    id_espe Especialidad   Horario Lunes 22/11 Martes 23/11 Miercoles 24/11  \
0         1  Odontología  10:00:00    cambiado   disponible      disponible   
1         1  Odontología  11:00:00  disponible   disponible      disponible   
2         1  Odontología  12:00:00  disponible   disponible      disponible   
3         1  Odontología  13:00:00  disponible   disponible      disponible   
4         1  Odontología  14:00:00  disponible   disponible      disponible   
..      ...          ...       ...         ...          ...             ...   
63        5    Pediatría  13:20:00  disponible          NaN      disponible   
64        5    Pediatría  14:00:00  disponible          NaN      disponible   
65        5    Pediatría  14:40:00  disponible          NaN      disponible   
66        5    